In [ ]:
from simulation.SimulationRunner import SimulationRunner, SimulationConfig, ModelType, TopologyType
from interactions.DialogueSimulation import DialogueSimulator
from dotenv import load_dotenv
load_dotenv()
from utils.log_config import setup_logging, print_to_log, log_config, log_simulation_start, generate_filename_with_date

## Create logs 

It is useful to create two log files. The first one, `app_<date>.log` logs all the interactions (requests and responses) with the OpenAI API. It will also log responses and requests made to ChatOllama if you are using Ollama to run local models. 

The second log file `simulation_<date>.log` logs all the simulation steps, and the actual interactions of the agents within a simulation. 

We create these logs first and then proceed to the simulation. 

In [ ]:
app_log_name = generate_filename_with_date("app")
sim_log_name = generate_filename_with_date("simulation")

logger = setup_logging(app_file_name=app_log_name, simulation_file_name=sim_log_name)



## Steps for Simulation 

1. Initialize the `SimulationRunner` object, and populate it with a `SimulationConfig` configuration
2. Inject the topic of discussion into conversation.
3. Let conversation evolve for desired number of steps

For this example, we create a Small World network with 10 agents who will discuss the issue of abortion rights. The simulation will run for `num_rounds=50` and we will use `gpt-3.5-turbo-16k` as the LLM. 

In [ ]:

sim_config = SimulationConfig(num_agents=10, topic = "Abortion Rights", num_rounds = 50, 
                              topology=TopologyType.SMALL_WORLD,
                             model_type=ModelType.GPT3BIS)


### Start logging the simulation

In [ ]:
log_simulation_start()

### We log the configuration for this simulation 

In [ ]:
log_config(sim_config.dict())

### Create the `SimulationRunner` object 

The creation of this object will initialize the graph and also the agents. The agent personas and preferences and opinions will also be generated using the LLM. 

In [ ]:
runner = SimulationRunner(config=sim_config, interaction_model=DialogueSimulator)

### Example persona for an agent 

In [ ]:
print(runner.interaction_model.agents[4].agent_description)

### To start the simulation, inject an empty message
Once an empty message is injected, an agent is picked at random. And then the message from this agent is transmitted to all other agents. For the next round, another agent is picked at random and the simulation continues. 

In [ ]:
runner.interaction_model.inject(None, "")
runner.run_simulation()

### Analyze agent messages 

`personal_message_history` contains the messages that this agent sent to other agents. 

The `history` object within `interaction_model` contains the history of all messages exchanged over the course of the simulation.


In [ ]:
agent_idx =4
for i, message in enumerate(runner.interaction_model.agents[agent_idx].personal_message_history):
    print(f'Message {i}: {message}')